In [44]:
import numpy as np 
import cv2 as cv 
import sklearn 
from matplotlib import pyplot as plt 
import tensorflow as tf 
import keras 
import seaborn
from IPython import display

In [45]:
def get_file_path(directory, filename):
    return os.path.join(directory, filename)

In [46]:
def plot_images(images):
    rows = 3
    cols = 3
    fig, axes = plt.subplots(rows, cols, figsize=(8, 8))
    #plotting images
    for i in range(rows):
        for j in range(cols):
            axes[i, j].imshow(images[i * cols + j], cmap='gray')
            axes[i, j].axis('off')

    plt.tight_layout()
    plt.show()

# Preparing Train

In [47]:
import os

In [ ]:
train_path=r'/kaggle/input/finger-print-dataset/Dataset/train'
files=os.listdir(train_path)
print('Train image data loading')
Train_data=[]
for file in files:
    image_path=get_file_path(train_path,file)
    img=cv.imread(image_path)
    Train_data.append(img)
print("Data loading Completed")
Train_data=np.array(Train_data)



Train image data loading


In [ ]:
#normalizing between[-1,1]
Train_data=Train_data/255

In [ ]:
plot_images(Train_data)

In [ ]:
enhance_path=r'/kaggle/input/finger-print-dataset/Dataset/enhace_train'
files=os.listdir(enhance_path)
print('Enhanced image data loading')
E_Train_data=[]
for file in files:
    image_path=get_file_path(enhance_path,file)
    img=cv.imread(image_path)
    img=cv.resize(img,(256,256))
    E_Train_data.append(img)
print("Data loading Completed")
E_Train_data=np.array(E_Train_data)




In [ ]:

E_Train_data=E_Train_data/255

In [ ]:
plot_images(E_Train_data)

In [ ]:
Train_data.shape

In [ ]:
Train_data=Train_data[4:]

In [ ]:
Train_data.shape

In [ ]:
E_Train_data.shape

# APPLYING GAN

In [ ]:
def generator_model():
    
     #generator model 
    input=tf.keras.layers.Input(shape=(512,512,3))
    x=tf.keras.layers.Conv2D(128,(5,5),strides=(1,1),padding='same',activation='relu',use_bias=False)(input)
    x=tf.keras.layers.Conv2D(128,(5,5),strides=(2,2),padding='same',activation='relu',use_bias=False)(x)
    
    x=tf.keras.layers.Conv2D(64,(5,5),strides=(1,1),padding='same',activation='relu',use_bias=False)(x)
    x=tf.keras.layers.Conv2D(64,(5,5),strides=(2,2),padding='same',activation='relu',use_bias=False)(x)
    
    x=tf.keras.layers.Conv2D(32,(5,5),strides=(1,1),padding='same',activation='relu',use_bias=False)(x)
    x=tf.keras.layers.Conv2D(32,(5,5),strides=(2,2),padding='same',activation='relu',use_bias=False)(x)
    
    x=tf.keras.layers.Conv2DTranspose(32,(5,5),strides=(1,1),padding='same',activation='relu')(x)
    x=tf.keras.layers.Conv2DTranspose(32,(5,5),strides=(2,2),padding='same',activation='relu',use_bias=False)(x)
    
    x=tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(1,1),padding='same',activation='relu')(x)
    x=tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',activation='relu',use_bias=False)(x)
    
    output=tf.keras.layers.Conv2DTranspose(3,(5,5),strides=(1,1),padding='same',use_bias=False ,activation='sigmoid')(x)

    return tf.keras.Model(inputs=input,outputs=output)


In [ ]:
generator=generator_model() 
generator.summary()

In [ ]:
#use for a single instance
img_batch = np.expand_dims(Train_data[0], axis=0)
generated_image=generator(img_batch,training=False) 
plt.imshow(generated_image[0],cmap='gray')

In [ ]:
def discriminator_model():

    input=tf.keras.layers.Input(shape=(256,256,3))
    x=tf.keras.layers.Conv2D(32,(5,5),strides=(2,2),padding='same')(input)
    x=tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x=tf.keras.layers.Dropout(0.3)(x)
    
    x=tf.keras.layers.Conv2D(64,(5,5),strides=(2,2),padding='same')(x)
    x=tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x=tf.keras.layers.Dropout(0.3)(x)

    x=tf.keras.layers.Conv2D(128,(5,5),strides=(2,2),padding='same')(x)
    x=tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x=tf.keras.layers.Dropout(0.3)(x)

    x=tf.keras.layers.Flatten()(x)
    output=tf.keras.layers.Dense(1,activation='sigmoid')(x)

    return tf.keras.Model(inputs=input,outputs=output)

In [ ]:
discriminator=discriminator_model()
discriminator.summary()

In [ ]:
decision=discriminator(generated_image)
print(decision)

# Create Custom Loss Functions

# Adversial Loss

In [ ]:
cross_entropy=tf.keras.losses.BinaryCrossentropy()

In [ ]:
def discriminator_loss(real_output,fake_output):
    real_loss=cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss=cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss=real_loss+fake_loss 
    return total_loss 

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

# Reconstruction Loss

In [ ]:
import tensorflow as tf

def reconstruction_loss(image1, image2):
    mse_total = mse(image1, image2)
    return mse_total / tf.cast(tf.shape(image1)[0], tf.float32)  # Divide by batch size

def mse(image1, image2):
    image1 = tf.cast(image1, tf.float32)
    image2 = tf.cast(image2, tf.float32)
    # Cast image1 to float32
    mse = tf.reduce_mean(tf.square(image1 - image2))
    return mse


# Orientation Loss

In [ ]:
def orientation_loss(image1,image2):
    ori_total=0 
    for i,j in zip(image1,image2):
        ori_total+=orientation(i,j)
    return ori_total/len(image1)
    

def orientation(image1,image2):
    detector = cv2.ORB_create()
    keypoints1, descriptors1 = detector.detectAndCompute(image1, None)
    keypoints2, descriptors2 = detector.detectAndCompute(image2, None)

    # Match keypoints between the images
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2)

    orientation_difference = 0
    for match in matches:
        kp1 = keypoints1[match.queryIdx]
        kp2 = keypoints2[match.trainIdx]
        orientation_difference += abs(kp1.angle - kp2.angle)

    average_orientation_difference = orientation_difference / len(matches)
    
    return average_orientation_difference



In [ ]:
gen_optimezer=tf.keras.optimizers.Adam(1e-4)
disc_optimezer=tf.keras.optimizers.Adam(1e-4)

In [ ]:
@tf.function 
# causes the function to be compiled 
def train_step(real_images,images,l1):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images=generator(real_images,training=True)

        real_output=discriminator(images,training=True)
        fake_output=discriminator(generated_images,training=True)
        
#         gen_loss=generator_loss(fake_output)+(l1*reconstruction_loss(images, generated_images))+(l2*orientation_loss(images, generated_images))
        gen_loss=generator_loss(fake_output)+(l1*reconstruction_loss(images, generated_images))
        disc_loss=discriminator_loss(real_output,fake_output)
        

    gradients_of_generator=gen_tape.gradient(gen_loss,generator.trainable_variables)
    gradients_of_discriminator=disc_tape.gradient(disc_loss,discriminator.trainable_variables)

    gen_optimezer.apply_gradients(zip(gradients_of_generator,generator.trainable_variables))
    disc_optimezer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(9, 9))

  for i in range(predictions.shape[0]):
      plt.subplot(3, 3, i+1)
      plt.imshow(predictions[i, :, :, 0] * 255, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
import tensorflow as tf
import numpy as np

def train(dataset, dataset2, epochs, batch_size,test):
    num_batches = len(dataset) // batch_size
    
    for epoch in range(epochs):
        print("Epoch {} started".format(epoch + 1))
        for batch_index in range(num_batches):
            start_index = batch_index * batch_size
            end_index = start_index + batch_size
            image_batch = dataset[start_index:end_index]
            enhanced_batch = dataset2[start_index:end_index]
            train_step(image_batch, enhanced_batch,0.5)
        
        # Display generated images after each epoch
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, test)
        
        print("Epoch {} completed".format(epoch + 1))
    
    # Display final generated images
    display.clear_output(wait=True)
    generate_and_save_images(generator, epoch, test)



In [ ]:
# train_images=tf.expand_dims(train_images,axis=1)

In [ ]:
test_path=r'/kaggle/input/finger-print-dataset/Dataset/test'
files=os.listdir(test_path)
print('Test image data loading')
Test_data=[]
for file in files:
    image_path=get_file_path(test_path,file)
    img=cv.imread(image_path)
    Test_data.append(img)
print("Data loading Completed")
Test_data=np.array(Test_data)

In [ ]:
Test_data.shape

In [ ]:
plot_images(Test_data)

In [ ]:
# Test_data=Test_data/255

In [ ]:
test_sample=Test_data[:9]
EPOCHS=10
batch_size=10
train(Train_data,E_Train_data,EPOCHS,batch_size,test_sample)

In [ ]:
# image_path=r'/kaggle/working/image_at_epoch_0007.png'
# display.clear_output(wait=True)
# img=cv.imread(image_path)
# plt.imshow(img)